In [5]:
import pandas as pd
import numpy as np

from keras import layers, models
from keras import regularizers
from keras.layers.normalization import BatchNormalization

/Users/jinli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
crowd_data = pd.read_pickle('../data/processed_data_crowd.pkl')

In [3]:
crowd_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393022 entries, 489 to 222446005
Data columns (total 4 columns):
project_id    393022 non-null int32
src           393022 non-null object
attribute     393022 non-null category
label         393022 non-null category
dtypes: category(2), int32(1), object(1)
memory usage: 8.2+ MB


In [4]:
crowd_data.head()

,project_id,src,attribute,label
mid,,,,
489,40030,https://mir-s3-cdn-cf.behance.net/project_modu...,emotion_peaceful,positive
1053,40041,https://mir-s3-cdn-cf.behance.net/project_modu...,emotion_peaceful,unsure
1065,40041,https://mir-s3-cdn-cf.behance.net/project_modu...,emotion_gloomy,unsure
1067,40043,https://mir-s3-cdn-cf.behance.net/project_modu...,emotion_peaceful,unsure
1247,40053,https://mir-s3-cdn-cf.behance.net/project_modu...,media_oilpaint,negative


In [7]:
batch_size = 100
epochs = 30

In [36]:
from keras import Input
from keras.models import Model


def create_model():
    input_tensor = Input(shape=(128, 128, 3))

    x = layers.Conv2D(
        32,
        (6, 6),
        activation='relu',
    )(input_tensor)

    x = layers.MaxPool2D(
        (2, 2),
        padding="same"
    )(x)

    x = BatchNormalization(
    )(x)

    x = layers.SeparableConv2D(
        64,
        3,
        activation='relu',
    )(x)

    x = layers.MaxPool2D(
        (2, 2),
        padding="same"
    )(x)

    x = BatchNormalization(
    )(x)

    x = layers.SeparableConv2D(
        128,
        3,
        activation='relu',
    )(x)

    x = layers.MaxPool2D(
        (2, 2),
        padding="same"
    )(x)

    x = BatchNormalization(
    )(x)
    #x = inception_module (x)

    x = layers.Flatten(
    )(x)

    x = layers.Dense(
        128,
        activation='relu',
        kernel_regularizer=regularizers.l2(0.001),
        #kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),
    )(x)

    x = layers.Dropout(
        0.5,
        name="Drop-Out"
    )(x)

    water_color_pred = layers.Dense(
        100,
        activation='softmax',
        name='water_color'
    )(x)

    oil_paint_pred = layers.Dense(
        3,
        activation='softmax',
        name='oil_paint'
    )(x)

#     model = Model(
#         input_tensor,
#         [water_color_pred, oil_paint_pred]
#     )
    model = Model(
        input_tensor,
        [oil_paint_pred]
    )
    return model


model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 123, 123, 32)      3488      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 62, 62, 32)        128       
_________________________________________________________________
separable_conv2d_13 (Separab (None, 60, 60, 64)        2400      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 30, 30, 64)        256       
__________

In [37]:
# data augmentation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=30,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(
    rescale=1./255,
)

directory = '../data/images/media_oilpaint/'
train_dir = directory + 'train'
valid_dir = directory + 'valid'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    batch_size=20,
    class_mode="categorical"
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(128,128),
    batch_size=20,
    class_mode="categorical"
)

Found 475 images belonging to 3 classes.
Found 470 images belonging to 3 classes.


In [38]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, \
    EarlyStopping, ReduceLROnPlateau, TensorBoard

weight_path = "{}_weights.best.hdf5".format('CNN2')

checkpoint = ModelCheckpoint(
    weight_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=True
)

# tensorboard = TensorBoard(
#     log_dir='Logs',
    
#     # ValueError: If printing histograms, validation_data must be provided, 
#     #and cannot be a generator.
#     #histogram_freq=5, # records activation histogram every n epoch
# )

reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=1, 
    verbose=1,
    mode='min',
    min_delta=0.0001, 
    cooldown=2, 
    min_lr=1e-7
)


def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate


lrate = LearningRateScheduler(step_decay)  # can place this in call_backs_list

early = EarlyStopping(
    monitor="val_loss",
    #monitor='acc'
    mode="min",
    verbose=2,
    # training is interrupted when the monitor argument stops improving after n steps
    patience=5
)

callbacks_list = [checkpoint, early, reduceLROnPlat]

In [39]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [40]:
history2 = model.fit_generator(
    train_generator,
    validation_data=valid_generator,
#     steps_per_epoch=round(len(train_images) / batch_size),
    steps_per_epoch=20,
    epochs=epochs,
#     validation_steps=round(len(valid_images) / batch_size),
    validation_steps=20,
    callbacks=callbacks_list
)

Epoch 1/30
20/20 [==============================] - 35s 2s/step - loss: 5.6439 - acc: 0.6158 - val_loss: 6.6663 - val_acc: 0.5717

Epoch 00001: val_loss improved from inf to 6.66628, saving model to CNN2_weights.best.hdf5
Epoch 2/30
20/20 [==============================] - 34s 2s/step - loss: 5.6026 - acc: 0.6456 - val_loss: 5.7092 - val_acc: 0.6393

Epoch 00002: val_loss improved from 6.66628 to 5.70916, saving model to CNN2_weights.best.hdf5
Epoch 3/30
20/20 [==============================] - 30s 1s/step - loss: 5.4993 - acc: 0.6511 - val_loss: 5.4340 - val_acc: 0.6769

Epoch 00003: val_loss improved from 5.70916 to 5.43398, saving model to CNN2_weights.best.hdf5
Epoch 4/30
20/20 [==============================] - 27s 1s/step - loss: 6.0468 - acc: 0.6223 - val_loss: 5.8426 - val_acc: 0.6632

Epoch 00004: val_loss did not improve from 5.43398

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/30
20/20 [==============================] - 27s 1s/ste

In [46]:
import os, os.path
# print (len([name for name in os.listdir(train_dir) if os.path.isfile(name)]))
neg_len_dir = len (os.listdir(train_dir + '/negative'))
pos_len_dir = len (os.listdir(train_dir + '/positive'))
uns_len_dir = len (os.listdir(train_dir + '/unsure'))

In [48]:
print (neg_len_dir, pos_len_dir, uns_len_dir)

245 86 144


In [49]:
245 / (245 + 86 + 144)

0.5157894736842106